In [7]:
def export_result(df: DataFrame, result_path: str, show_rows: int = 20,
                  partition_column: str = None, num_partitions: int = None) -> None:
    """
    Виводить результат в консоль, рахує кількість записів та експортує DataFrame у CSV з партиціонуванням.

    Args:
        df: Результуючий DataFrame.
        result_path: Шлях до директорії, де буде збережено результат.
        show_rows: Кількість рядків для виводу в консоль.
        partition_column: Колонка, за якою слід партиціонувати вихідні дані.
        num_partitions: Кількість партицій для запису (якщо не вказана колонка розділення).
    """
    print(f"Результати для: {result_path}")
    df.show(show_rows, truncate=False)
    total_records = df.count()
    print(f"Total number of records in the result: {total_records}")

    # Конвертуємо масиви в рядки для коректного запису в CSV
    for col_name, col_type in df.dtypes:
        if col_type.startswith("array"):
            df = df.withColumn(col_name, F.concat_ws(", ", F.col(col_name)))

    # Налаштування запису з партиціонуванням
    writer = df.write.option("header", "true")

    # Спосіб 1: Партиціонування за вказаною колонкою
    if partition_column and partition_column in df.columns:
        print(f"Експорт результатів з партиціонуванням за колонкою: {partition_column}")
        writer.partitionBy(partition_column).csv(result_path, mode="overwrite")

    # Спосіб 2: Перерозподіл на вказану кількість партицій
    elif num_partitions and num_partitions > 0:
        print(f"Експорт результатів з {num_partitions} партиціями")
        df.repartition(num_partitions).write.option("header", "true").csv(result_path, mode="overwrite")

    # Спосіб 3: Автоматичне визначення оптимальної кількості партицій
    else:
        # Орієнтовний розмір партиції (128MB - типовий розмір блоку HDFS)
        target_size_mb = 128
        estimated_size_bytes = total_records * len(df.columns) * 100
        estimated_size_mb = estimated_size_bytes / (1024 * 1024)
        optimal_partitions = max(1, int(estimated_size_mb / target_size_mb))

        print(f"Експорт результатів з автоматично визначеними {optimal_partitions} партиціями")
        df.repartition(optimal_partitions).write.option("header", "true").csv(result_path, mode="overwrite")

In [ ]:
from pyspark.sql import SparkSession
import os
from pyspark.sql import DataFrame
from typing import Dict
from config import Config
import pyspark.sql.types as t
import pyspark.sql.functions as F
from pyspark.sql.window import Window

from schemas import (
    schema_title_basics,
    schema_title_episode,
    schema_title_crew,
    schema_title_akas,
    schema_title_ratings,
    schema_title_principals,
    schema_name_basics
)

from imdb_spark_utils import (
    initialize_spark,
    load_dataframe,
    transform_title_basics,
    transform_title_akas,
    transform_title_crew,
    transform_title_episode,
    correlation_runtime_rating,
    display_title_type_info,
    transform_title_ratings,
    transform_title_principals,
    transform_name_basics,
    export_result
)

spark_session = initialize_spark("IMDB Data Processing")


def check_pyspark() -> None:
    print("Initializing Spark Session...")
    spark = SparkSession.builder \
        .appName("IMDB Data Check") \
        .getOrCreate()

    print("Spark Session initialized.")
    print("Checking available files...")
    files = [f for f in os.listdir(Config.DATA_DIR) if f.endswith(Config.FILE_EXTENSION)]

    if not files:
        print("No TSV files found. Make sure data is downloaded and extracted.")
        return

    print("Found files:", files)

    sample_file = os.path.join(Config.DATA_DIR, files[0])
    print(f"Loading sample file: {sample_file}")

    df = spark.read.option("header", "true").option("sep", "\t").csv(sample_file)

    print("Schema of the loaded file:")
    df.printSchema()

    print("Showing first 5 rows:")
    df.show(5)

    print("PySpark check complete!")

    spark.stop()


def process_imdb_data() -> Dict[str, DataFrame]:
    dataframes = {}

    dataframes["basics"] = transform_title_basics(
        load_dataframe(spark_session, schema_title_basics, f"{Config.DATA_DIR}/title.basics{Config.FILE_EXTENSION}")
    )
    dataframes["akas"] = transform_title_akas(
        load_dataframe(spark_session, schema_title_akas, f"{Config.DATA_DIR}/title.akas{Config.FILE_EXTENSION}")
    )
    dataframes["crew"] = transform_title_crew(
        load_dataframe(spark_session, schema_title_crew, f"{Config.DATA_DIR}/title.crew{Config.FILE_EXTENSION}")
    )
    dataframes["episode"] = transform_title_episode(
        load_dataframe(spark_session, schema_title_episode, f"{Config.DATA_DIR}/title.episode{Config.FILE_EXTENSION}")
    )
    dataframes["ratings"] = transform_title_ratings(
        load_dataframe(spark_session, schema_title_ratings, f"{Config.DATA_DIR}/title.ratings{Config.FILE_EXTENSION}")
    )
    dataframes["principals"] = transform_title_principals(
        load_dataframe(spark_session, schema_title_principals, f"{Config.DATA_DIR}/title.principals{Config.FILE_EXTENSION}")
    )
    dataframes["name"] = transform_name_basics(
        load_dataframe(spark_session, schema_name_basics, f"{Config.DATA_DIR}/name.basics{Config.FILE_EXTENSION}")
    )

    # correlation between runtime and rating
    correlation_runtime_rating(dataframes["basics"], dataframes["ratings"])

    # categories of titleTypes (e.g. movie, tvSeries, short, video etc)
    display_title_type_info(dataframes["basics"])

    return dataframes


dataframes = process_imdb_data()

Correlation between runtime and average rating: -0.001390673371675943
TitleType counts and average runtime:
+------------+-------+------------------+
|titleType   |count  |average_runtime   |
+------------+-------+------------------+
|tvSeries    |280071 |80.99930614960554 |
|tvMiniSeries|61022  |74.89654025282768 |
|tvMovie     |150638 |71.7865250220512  |
|tvEpisode   |8917320|38.38889760316851 |
|movie       |712105 |89.66853104173349 |
|tvSpecial   |52099  |86.57129751505724 |
|video       |309195 |69.9779210545168  |
|videoGame   |42613  |104.46651785714286|
|tvShort     |10619  |12.242874045390986|
|short       |1055152|13.090382470884505|
|tvPilot     |1      |0.0               |
+------------+-------+------------------+



In [ ]:
def execute_analytical_requests(dataframes: Dict[str, DataFrame], results_dir: str) -> None:
    """
    Execute 6 analytical requests on the IMDB dataset, ensuring all requirements are met.

    Args:
        dataframes: Dictionary containing the loaded dataframes
        results_dir: Directory path to save results
    """
    print("\n" + "="*50)
    print("EXECUTING ANALYTICAL REQUESTS (ENSURING ALL REQUIREMENTS)")
    print("="*50)

    # Request 1: Find top 10 highest-rated movies with at least 5,000 votes released after 2000 (uses multiple filters and join)
    request1_result = find_top_rated_recent_movies(dataframes["basics"], dataframes["ratings"])
    export_result(request1_result, f"{results_dir}/top_rated_recent_movies",
                  title="Top 10 Highest-Rated Movies (>= 5000 votes, > 2000)")

    # Request 2: Find actors who starred in movies with average rating > 8.2 and have at least 3 known for titles (uses multiple filters and joins)
    request2_result = find_popular_actors_in_highly_rated_movies(dataframes["basics"], dataframes["ratings"],
                                                                 dataframes["principals"], dataframes["name"])
    export_result(request2_result, f"{results_dir}/popular_actors_highly_rated",
                  title="Actors in Highly Rated Movies (>= 3 known titles)")

    # Request 3: Calculate the average number of votes and average rating for each genre (uses group by)
    request3_result = calculate_genre_popularity_rating(dataframes["basics"], dataframes["ratings"])
    export_result(request3_result, f"{results_dir}/genre_popularity_rating",
                  title="Average Votes and Rating by Genre")

    # Request 4: Find the average runtime for each title type, filtering out title types with fewer than 1000 entries (uses group by and filter)
    request4_result = average_runtime_by_title_type(dataframes["basics"])
    export_result(request4_result, f"{results_dir}/avg_runtime_by_type",
                  title="Average Runtime by Title Type (>= 1000 entries)")

    # Request 5: Rank movies within each genre based on their average rating (uses window functions and join)
    request5_result = rank_movies_by_rating_within_genre(dataframes["basics"], dataframes["ratings"])
    export_result(request5_result, f"{results_dir}/ranked_movies_by_genre",
                  title="Movies Ranked by Rating within Genre")

    # Request 6: Calculate the cumulative average rating of episodes within each TV series, ordered by season and episode number (uses window functions and joins)
    request6_result = cumulative_avg_rating_tv_series(dataframes["basics"], dataframes["ratings"], dataframes["episode"])
    export_result(request6_result, f"{results_dir}/cumulative_avg_rating_series",
                  title="Cumulative Average Rating of Episodes within TV Series")


def find_top_rated_recent_movies(basics_df: DataFrame, ratings_df: DataFrame) -> DataFrame:
    """
    Request 1: Find top 10 highest-rated movies with at least 5,000 votes released after 2000.
    """
    print("\nExecuting Request 1: Top 10 Highest-Rated Recent Movies")
    movies_df = basics_df.filter(F.col("titleType") == "movie")

    result = movies_df.join(ratings_df, "tconst") \
                     .filter((F.col("numVotes") >= 5000) & (F.col("startYear") > 2000)) \
                     .select("tconst", "primaryTitle", "startYear", "genres", "averageRating", "numVotes") \
                     .orderBy(F.col("averageRating").desc()) \
                     .limit(10)

    return result


def find_popular_actors_in_highly_rated_movies(basics_df: DataFrame, ratings_df: DataFrame,
                                                principals_df: DataFrame, name_df: DataFrame) -> DataFrame:
    """
    Request 2: Find actors who starred in movies with average rating > 8.2 and have at least 3 known for titles.
    """
    print("\nExecuting Request 2: Popular Actors in Highly Rated Movies")
    highly_rated_movies = basics_df.join(ratings_df, "tconst") \
                                    .filter((F.col("titleType") == "movie") & (F.col("averageRating") > 8.2)) \
                                    .select("tconst")

    actors_in_movies = principals_df.join(highly_rated_movies, "tconst") \
                                     .filter(F.col("category").isin("actor", "actress")) \
                                     .select("nconst", "tconst")

    actor_known_titles = name_df.filter(F.size(F.col("knownForTitles")) >= 3) \
                                .select("nconst", "primaryName")

    result = actors_in_movies.join(actor_known_titles, "nconst") \
                             .groupBy("nconst", "primaryName") \
                             .agg(F.countDistinct("tconst").alias("movies_starred_in")) \
                             .orderBy(F.col("movies_starred_in").desc())

    return result


def calculate_genre_popularity_rating(basics_df: DataFrame, ratings_df: DataFrame) -> DataFrame:
    """
    Request 3: Calculate the average number of votes and average rating for each genre.
    """
    print("\nExecuting Request 3: Average Votes and Rating by Genre")
    movies_with_ratings = basics_df.join(ratings_df, "tconst")
    exploded_genres = movies_with_ratings.select(F.explode("genres").alias("genre"), "averageRating", "numVotes")

    result = exploded_genres.groupBy("genre") \
                            .agg(F.avg("averageRating").alias("avg_rating"),
                                 F.avg("numVotes").alias("avg_votes")) \
                            .orderBy(F.col("avg_rating").desc())

    return result


def average_runtime_by_title_type(basics_df: DataFrame) -> DataFrame:
    """
    Request 4: Find the average runtime for each title type, filtering out title types with fewer than 1000 entries.
    """
    print("\nExecuting Request 4: Average Runtime by Title Type (>= 1000 entries)")
    title_type_counts = basics_df.groupBy("titleType").count().filter(F.col("count") >= 1000)

    result = basics_df.join(title_type_counts, "titleType") \
                      .groupBy("titleType") \
                      .agg(F.avg("runtimeMinutes").alias("avg_runtime")) \
                      .orderBy(F.col("avg_runtime").desc())

    return result


def rank_movies_by_rating_within_genre(basics_df: DataFrame, ratings_df: DataFrame) -> DataFrame:
    """
    Request 5: Rank movies within each genre based on their average rating.
    """
    print("\nExecuting Request 5: Rank Movies by Rating within Genre")
    movies_with_ratings = basics_df.join(ratings_df, "tconst")
    exploded_genres = movies_with_ratings.select("tconst", "primaryTitle", F.explode("genres").alias("genre"), "averageRating")

    window_spec = Window.partitionBy("genre").orderBy(F.col("averageRating").desc())

    result = exploded_genres.withColumn("rank_within_genre", F.rank().over(window_spec)) \
                            .orderBy("genre", "rank_within_genre")

    return result


def cumulative_avg_rating_tv_series(basics_df: DataFrame, ratings_df: DataFrame, episode_df: DataFrame) -> DataFrame:
    """
    Request 6: Calculate the cumulative average rating of episodes within each TV series,
               ordered by season and episode number (uses window functions and joins).
    """
    print("\nExecuting Request 6: Cumulative Average Rating of Episodes within TV Series")
    tv_episodes = basics_df.filter(F.col("titleType") == "tvEpisode")
    episodes_with_ratings = tv_episodes.join(ratings_df, "tconst") \
                                       .join(episode_df, "tconst") \
                                       .join(basics_df.alias("series"), F.col("parentTconst") == F.col("series.tconst")) \
                                       .select(F.col("series.primaryTitle").alias("series_title"),
                                               "seasonNumber", "episodeNumber", "averageRating") \
                                       .orderBy("series_title", "seasonNumber", "episodeNumber")

    window_spec = Window.partitionBy("series_title").orderBy("seasonNumber", "episodeNumber")

    result = episodes_with_ratings.withColumn("cumulative_avg_rating", F.avg("averageRating").over(window_spec))

    return result

In [19]:
results_dir = 'results'
execute_analytical_requests(dataframes, results_dir)


EXECUTING ANALYTICAL REQUESTS (ENSURING ALL REQUIREMENTS)

Executing Request 1: Top 10 Highest-Rated Recent Movies
Results for: Top 10 Highest-Rated Movies (>= 5000 votes, > 2000)
+----------+-------------------------------------------------+---------+------------------------------+-------------+--------+
|tconst    |primaryTitle                                     |startYear|genres                        |averageRating|numVotes|
+----------+-------------------------------------------------+---------+------------------------------+-------------+--------+
|tt21272942|The Strangers' Case                              |2024     |[Drama]                       |9.3          |8283    |
|tt33175825|Attack on Titan the Movie: The Last Attack       |2024     |[Action, Adventure, Animation]|9.2          |15189   |
|tt1114271 |Thursday                                         |2006     |[Action, Adventure, Crime]    |9.1          |5256    |
|tt31853193|Gundi: Legend of Love                        